In [2]:
!pip install torchtext

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 15.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
train_df = pd.read_csv("D:\\telelyzerv2\\experiment\\processed_ds.csv")

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Sample data loading - replace with your actual data
# train_df = pd.DataFrame({
#     "text": ["Example sentence 1", "Example sentence 2"],  # Replace with your actual data
#     "label": [0, 1]  # Replace with actual labels
# })

# Tokenization and vocabulary creation
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

# Build vocabulary from the training dataset
vocab = build_vocab_from_iterator(yield_tokens(train_df['text']), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])  # Assign <unk> to unknown tokens

# Prepare data
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return torch.tensor(vocab(tokenizer(text)), dtype=torch.long), label

# Splitting data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df["text"], train_df["label"], test_size=0.2, random_state=42
)

# Create datasets
train_dataset = TextDataset(train_texts.tolist(), train_labels.tolist())
val_dataset = TextDataset(val_texts.tolist(), val_labels.tolist())

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Define the Bi-GRU model
class BiGRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(BiGRU, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # 2 for bidirection
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.gru(x)
        x = self.dropout(x[:, -1, :])  # Get the last output of the GRU
        return self.fc(x)

# Hyperparameters
vocab_size = len(vocab)
embedding_dim = 100
hidden_dim = 64
output_dim = 2  # Binary classification
num_epochs = 5
learning_rate = 0.001

# Initialize model, loss function, and optimizer
model = BiGRU(vocab_size, embedding_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training function
def train_model(model, train_loader, criterion, optimizer):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.numpy())
            true_labels.extend(labels.numpy())
    return true_labels, predictions

# Training loop
for epoch in range(num_epochs):
    train_model(model, train_loader, criterion, optimizer)
    true_labels, predictions = evaluate_model(model, val_loader)
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average="binary")
    print(f"Epoch {epoch + 1}/{num_epochs} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

# Save the model
torch.save(model.state_dict(), 'bi_gru_model.pth')
print("Training complete and model saved.")


c:\Python\Lib\site-packages\torchtext\__init__.py:7: SyntaxWarning: invalid escape sequence '\ '
  "\n/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ \n"


OSError: [WinError 127] The specified procedure could not be found